<a href="https://colab.research.google.com/github/resquilleur/TestWorkNeowoxCC/blob/master/DataPrepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# git clone

%cd TestWorkNeowoxCC

# Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
import pymorphy2
import pickle

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Загрузка и обработка наборов данных

In [ ]:
# загрузим набор данных
data_train = pd.read_csv('data/for_test_train.csv').drop(columns=['Unnamed: 0'])
# посмотрим кол-во экземпляров по классам
data_train.groupby('label').count()

,message
label,
tag_do_not_call,85
tag_no,122
tag_no_now,1
tag_not_now,67
tag_what_company,36
tag_who_is,50
tag_yes,191


In [ ]:
# видим, что у нас есть всего один экземпляр класса tag_no_now, по смыслу подходит к тегу tag_not_now
# я решил объеденить эти классы
data_train[data_train['label'] == 'tag_no_now']

,message,label
543,"ты шутишь, у меня нет времени!",tag_no_now


In [ ]:
# получим индекс нужного тега и переименуем его в правильный тег
idx = (data_train['label'] == 'tag_no_now')
data_train.loc[idx, 'label'] = 'tag_not_now'
data_train.groupby('label').count()

,message
label,
tag_do_not_call,85
tag_no,122
tag_not_now,68
tag_what_company,36
tag_who_is,50
tag_yes,191


In [ ]:
# загрузим проверочный набор данных
data_valid = pd.read_csv('data/for_test_valid.csv', sep=';') # обязательно использовать такой разделитель, иначе не загрузится.
data_valid # проверочный набор

,Unnamed: 0,message
0,106,зачем это надо
1,162,ну
2,602,какая организация?
3,32,во имя чего?
4,642,вы тут?
5,419,да ну вас!
6,854,"приветствую, а что за организация?"
7,59,да хорошо
8,219,с какой радости?
9,112,и что дальше?


## Аугментация данных

In [ ]:
# видим дисбаланс классов и в целом датасет небольшой, сделаем аугментацию для повышения качество распознования классов
aug_data_train = pd.DataFrame() # создадим пустой датафрейм
labels = data_train['label'].unique() # список уникальных меток классов

# сделаем так, чтобы обязательно попали все примеры, и добавим сгенерированные бутстрапом, так чтобы в сумме оказалось 200 шт каждого класса
for label in labels:
    n_samples = 0
    aug_data_train = pd.concat([aug_data_train, data_train[data_train['label'] == label]], ignore_index=True)
    if label not in ['tag_yes', 'tag_no']:
        n_samples = 200 - data_train[data_train['label'] == label].count()[0]
    aug_data_train = pd.concat([aug_data_train, data_train[data_train['label'] == label].sample(n=n_samples, replace=True, random_state=42)], ignore_index=True)

aug_data_train

,message,label
0,будьте здоровы!,tag_no
1,не имеется,tag_no
2,ни фига,tag_no
3,вы бредите,tag_no
4,"нет, нашел дурака",tag_no
...,...,...
1108,скажите по новой,tag_who_is
1109,"да, алло, я не расслышал",tag_who_is
1110,"да, алло, я не расслышал",tag_who_is
1111,это кто?,tag_who_is


## Формируем наборы

In [ ]:
# неагментированные данные
x_train_not_aug = data_train['message'].values
y_train_not_aug = data_train['label'].values

# формируем x_valid
x_valid = data_valid['message'].values

# формируем x_train, y_train
x_train = aug_data_train['message'].values
y_train = aug_data_train['label'].values

# сделаем one_hot_encoding для набора классов
ohe_ytrain = OneHotEncoder()
y_train = ohe_ytrain.fit_transform(y_train.reshape(-1,1)).toarray() # аугментированный набор
y_train[0]

array([0., 1., 0., 0., 0., 0.])

In [ ]:
# функция приведения к нормальной форме слова
def data_normalized(x_data):
    x_data_normal = []
    for frase in x_data:
        new_frase = ''
        words = frase.split()
        for word in words:
            word = word.replace('!', '').replace(',', '').replace('?', '')
            new_frase += f' {ma.normal_forms(word)[0]}'
        x_data_normal.append(new_frase)
    return np.array(x_data_normal)

In [ ]:
# создаем анализатор слов
ma = pymorphy2.MorphAnalyzer()

# далее у нас будет два варианта набора данных с нормой и без
x_valid_norm = data_normalized(x_valid)
x_train_norm = data_normalized(x_train)

In [ ]:
# токенизируем слова без нормирования
tokenizer = Tokenizer()
tokenizer.fit_on_texts(np.concatenate((x_train, x_valid), axis=0)) # обучим на всех словах, что у нас есть

tokenizer_norm = Tokenizer()
tokenizer_norm.fit_on_texts(np.concatenate((x_train_norm, x_valid_norm), axis=0)) # обучим на всех словах, что у нас есть

# сохраним токенайзеры для вывода результатов
with open('data/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('data/tokenizer_norm.pickle', 'wb') as handle:
    pickle.dump(tokenizer_norm, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# индексы в последовательность, дополним нулями перед последовательностью
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_train_pad_seq = pad_sequences(x_train_seq, padding='pre')

x_train_seq_norm = tokenizer_norm.texts_to_sequences(x_train_norm)
x_train_pad_seq_norm = pad_sequences(x_train_seq_norm, padding='pre')

x_valid_seq = tokenizer.texts_to_sequences(x_valid)
x_valid_pad_seq = pad_sequences(x_valid_seq, padding='pre')

x_valid_seq_norm = tokenizer_norm.texts_to_sequences(x_valid_norm)
x_valid_pad_seq_norm = pad_sequences(x_valid_seq_norm, padding='pre')

In [ ]:
# посмотрим что получилось
print(x_train_pad_seq[0])
print(x_train[0])

print(x_train_pad_seq_norm[0])
print(x_train_norm[0])

[  0   0   0   0   0   0   0 331 332]
будьте здоровы!
[  0   0   0   0   0   0   0 101 207]
 быть здоровый


In [ ]:
# y_train общий сохраним для семплирования
y_train_full = y_train.copy()

In [ ]:
y_train_full.shape

(1113, 6)

In [ ]:
# разобьем наборы на тестовый и тренировочный
x_train, x_test, y_train, y_test = train_test_split(x_train_pad_seq, y_train_full, test_size=0.2, stratify=y_train_full) # stratify позволяет сохранить баланс классов
x_train_norm, x_test_norm, y_train_norm, y_test_norm = train_test_split(x_train_pad_seq_norm, y_train_full, test_size=0.2, stratify=y_train_full) # stratify позволяет сохранить баланс классов

## Сохранение наборов для использывания в обучение

Подготовил наборы:
* x_train, x_test, y_train, y_test - это аугментированные, но *не* нормализованные наборы

* x_train_norm, x_test_norm, y_train_norm, y_test_norm - это аугментированные, нормализованные наборы

* x_valid_pad_seq, x_valid_pad_seq_norm - аналогично проверочные наборы

In [ ]:
numpy_data_list = [x_train, x_test, y_train, y_test, x_train_norm, x_test_norm, y_train_norm, y_test_norm, x_valid_pad_seq, x_valid_pad_seq_norm]
list_name = ['x_train', 'x_test', 'y_train', 'y_test', 'x_train_norm', 'x_test_norm', 'y_train_norm', 'y_test_norm', 'x_valid_pad_seq', 'x_valid_pad_seq_norm']

In [ ]:
# функция записи
def numpy_save(data, list_name):
    with open(f'data/{list_name}.npy', 'wb') as f:
        np.save(f, data)

In [ ]:
# сохраним все на диск
for np_data, name in zip(numpy_data_list, list_name):
    numpy_save(np_data, name)